In [ ]:
require "awesome_print"

# Framework Classes
require_relative "FrameworkClasses/Users"
require_relative "FrameworkClasses/Auth"
require_relative "FrameworkClasses/Permissions"
require_relative "FrameworkClasses/RolesToPermissions"
require_relative "FrameworkClasses/PermissionReader"
require_relative "FrameworkClasses/Audit"

# Business Logic Classes
require_relative "BusinessLogicClasses/SalesOrder"
require_relative "BusinessLogicClasses/Invoice"

# Modules
require_relative "Modules/CommonClassMethods"
require_relative "Modules/CommonCreateMethods"
require_relative "Modules/CommonReadMethod"
require_relative "Modules/CommonUpdateMethod"
require_relative "Modules/CommonPrintMethod"

In [15]:
module CommonInstanceMethods
    include CommonCreateMethods
    include CommonReadMethod
    include CommonUpdateMethod
    include CommonPrintMethod
end

#<Class:0x00000220366b6cc0>::CommonInstanceMethods

In [18]:
Users.create_or_update("Admin", "god")
Users.create_or_update("Kate", "123")
Users.create_or_update("Fluffy", "dog")

Users.get

User "Admin" was created or updated
User "Kate" was created or updated
User "Fluffy" was created or updated


["Admin", "Kate", "Fluffy"]

In [19]:
Permissions.add("Admin", "ASD", :Administrator)
Permissions.add("Kate", "ASD", :InvoicingSpecialist)
Permissions.add("Fluffy", "ASD", :Dog)

[:Dog]

In [20]:
RolesToPermissions.set( :Administrator,       :Invoice,    :do_create,       [:create])
RolesToPermissions.set( :Administrator,       :Invoice,    :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :Invoice,    :update,          [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :Invoice,    :do_delete,       [:delete])

RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :do_create,       [:create])
RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :update,          [:amount, :paid_amount, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist,  :Invoice,   :do_delete,       [:delete])

# --- #

RolesToPermissions.set( :Administrator,       :SalesOrder, :do_create,       [:create])
RolesToPermissions.set( :Administrator,       :SalesOrder, :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :SalesOrder, :update,          [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :SalesOrder, :do_delete,       [:amount, :paid_amount, :client, :name, :is_paid])

RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :do_create,       [:create])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :update,          [:amount, :recognized])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :do_delete,       [])


[]

In [21]:
Auth.login("Admin", "god")

Logged in as Admin


In [ ]:
Auth.login("Kate", "123")

In [22]:
so1 = SalesOrder.new ( 
    {name: "SO-1", 
    amount: 100,
    client: "ASD", 
    recognized: false} )

SalesOrder was created:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}


#<#<Class:0x00000220366b6cc0>::SalesOrder:0x0000022035df4358 @entity=:SalesOrder, @values={:name=>"SO-1", :amount=>100, :client=>"ASD", :recognized=>false}>

In [23]:
so1 = SalesOrder.new ( 
    {name: "so1", 
    amount: 100,
    client: "ASD", 
    recognized: false} )

SalesOrder was created:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}


#<#<Class:0x00000220366b6cc0>::SalesOrder:0x0000022036499938 @entity=:SalesOrder, @values={:name=>"so1", :amount=>100, :client=>"ASD", :recognized=>false}>

In [24]:
SalesOrder::Log.print

Admin changed the following values for SO-1:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}
New values:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}

Admin changed the following values for so1:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}
New values:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}



In [27]:
invoice1 = so1.create_invoice("ASD-1")

Invoice was created:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}


#<#<Class:0x00000220366b6cc0>::Invoice:0x000002203661e1a0 @entity=:Invoice, @values={:name=>"ASD-1", :amount=>100, :paid_amount=>0, :client=>"ASD", :is_paid=>false}>

In [28]:
invoice1.update( amount: 120, name: "ASD-3"  )
#invoice1.read([:amount, :is_paid])

Invoice was updated. New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}


In [29]:
invoice1.pay(20)

Invoice was updated. New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 20,
         :client => "ASD",
        :is_paid => false
}


In [30]:
Invoice::Log.print

Admin changed the following values for ASD-1:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}
New values:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}

Admin changed the following values for ASD-3:
{
    :amount => 120,
      :name => "ASD-3"
}
New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}

Admin changed the following values for ASD-3:
{
    :paid_amount => 20
}
New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 20,
         :client => "ASD",
        :is_paid => false
}



In [31]:
Invoice::Log.select_entries(amount: 100, is_paid: false)

Admin changed the following values for ASD-1:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}
New values:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}



In [ ]:
Invoice::Log.clear_log

In [32]:
so1.print

# ---------- #

SalesOrder so1

Name: so1
Amount: 100
Client: ASD
Recognized: false

# ---------- #


In [33]:
invoice1.print

# ---------- #

Invoice ASD-3

Name: ASD-3
Amount: 120
Paid_amount: 20
Client: ASD
Is_paid: false

# ---------- #
